In [4]:
import cv2
import numpy as np
from scipy.signal import find_peaks

# Inputs the raw AFM image, binarizes the image and outputs the binarized image
class Preprocess:
    def __init__(self, directory):
        self.global_thresh_value = 350
        self.adaptive_thresh_window_size = 25
        self.adaptive_thresh_C = 2
        self.morph_kernel_size = 2
        self.morph_iterations = 1
        self.directory = directory

    def read_image(self):
        self.image_path = f'./raw_data/X_MN/{directory}.png'
        return cv2.imread(self.image_path, cv2.IMREAD_GRAYSCALE)

    def global_threshold(self, image):
        _, global_thresh_mask = cv2.threshold(image, self.global_thresh_value, 255, cv2.THRESH_BINARY)
        return global_thresh_mask

    def adaptive_threshold(self, image):
        adaptive_mask = cv2.adaptiveThreshold(
            image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,
            self.adaptive_thresh_window_size, self.adaptive_thresh_C)
        return adaptive_mask

    def combine_masks(self, global_mask, adaptive_mask):
        return cv2.bitwise_or(global_mask, adaptive_mask)

    def morphological_operations(self, combined_mask):
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (self.morph_kernel_size, self.morph_kernel_size))
        return cv2.morphologyEx(combined_mask, cv2.MORPH_OPEN, kernel, iterations=self.morph_iterations)

    def save_cleaned_image(self, cleaned_image):
        cv2.imwrite(self.output_path, cleaned_image)

    def process_image(self):
        image = self.read_image()
        global_mask = self.global_threshold(image)
        adaptive_mask = self.adaptive_threshold(image)
        combined_mask = self.combine_masks(global_mask, adaptive_mask)
        cleaned_image = self.morphological_operations(combined_mask)
        
        return cleaned_image
    
class AFMAnalyzer:
    def __init__(self, image, sample, prominence_value=5, distance_value=10):
        self.image = image
        self.sample = sample
        self.prominence_value = prominence_value
        self.distance_value = distance_value

    def radial_profile(self, data, center):
        y, x = indices(data.shape)
        r = np.sqrt((x - center[0])**2 + (y - center[1])**2)
        r = r.astype(int)
        tbin = np.bincount(r.ravel(), data.ravel())
        nr = np.bincount(r.ravel())
        radialprofile = tbin / nr
        return radialprofile

    def process_image(self):
        image = self.image
        mask = np.where(image == 0, 0, 1).astype(np.uint8)
        image_masked = cv2.multiply(image, mask)

        f = np.fft.fft2(image_masked)
        fshift = np.fft.fftshift(f)
        magnitude_spectrum = 20 * np.log(np.abs(fshift) + 1)

        center = (int(np.floor(fshift.shape[0] / 2)), int(np.floor(fshift.shape[1] / 2)))
        radial_dist = self.radial_profile(magnitude_spectrum, center)

        truncated_radial_dist = radial_dist[:100]
        peaks, _ = find_peaks(truncated_radial_dist, prominence=self.prominence_value, distance=self.distance_value)

        # Create a mask to filter out peaks outside the range [25, 35]
        mask = (peaks >= 25) & (peaks <= 35)

        # Filter the peaks using the mask
        filtered_peaks = peaks[mask]
        
        self.peaks = filtered_peaks

    def save_peak_positions(self, output_path='./outputs/domain_spacing_X.txt'):
        with open(output_path, 'a') as file:  # Open file in append mode
            for peak_position in self.peaks:
                file.write(f"Sample X_{self.sample}: {1000/peak_position}\n")

In [5]:
# Number related to your fist sample
sample = 1
# parent_path = os.getcwd()

#Create a loop for all samples
for i in range(41):
    # Output path for the cleaned image
    directory = f'X{sample}'
    # Create an instance of Preprocess and process the image
    binzarize_image = Preprocess(directory)
    # Call the instance to run the class
    binned_image = binzarize_image.process_image()

    # Define the variables for AFMAnalyzer
#     image = binned_image
    analyzer = AFMAnalyzer(binned_image, sample)
    analyzer.process_image()
    analyzer.save_peak_positions()
#     print("Peak positions saved to 'peak_positions.txt'")
    sample +=1

NameError: name 'indices' is not defined